<a href="https://colab.research.google.com/github/21Ovi/TensorFlow_/blob/main/06_NPL_in_TensorFlow_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in TensorFlow

NPL has the goal of deriving information out of natural language (could be sequences text to speech).

Another common term for NLP problems is sequence to sequence problem (seq2seq)

In [1]:
!nvidia-smi

Sun Aug 13 09:48:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py

# Import series of helper function for noteebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-08-13 09:48:29--  https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10116 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.88K  --.-KB/s    in 0s      

2023-08-13 09:48:29 (131 MB/s) - ‘helper_functions.py’ saved [10116/10116]



## Get a text dataset

The dataset we're going to using is Kaggle's introduction to NPL dataset (text samples of Tweets labelled as disaster or not disaster).

Resource : https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-08-13 09:48:32--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 74.125.196.128, 142.251.162.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-08-13 09:48:32 (121 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in.

But I prefer to get visualize straigt away.

So another way to do this is to use pandas.

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Suffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look loke?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples

for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text: 
@TemecaFreeman GM! I pray any attack of the enemy 2 derail ur destiny is blocked by the Lord &amp; that He floods ur life w/heavenly Blessings

---

Target: 1 (real disaster)
Text: 
24 killed in two simultaneous rail crash as acute floods derail the two trains #India #mumbai... http://t.co/4KBWPCmMbM

---

Target: 1 (real disaster)
Text: 
BBC News - India rail crash: Trains derail in Madhya Pradesh flash flood http://t.co/fU1Btuq1Et

---

Target: 0 (not real disaster)
Text: 
This is how we know #AllLivesMatter people are incredibly racist and only care to derail necessary conversations

---

Target: 1 (real disaster)
Text: 
Will @FoxNews continue to derail the Iran Nuclear Deal during tonight's #GOPDebate  oh yeah.

---



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
## Use train_train_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1, # use 10% of training data for validation set
                                                                             random_state=42)

In [12]:
# Check lengths
len(train_sentences), len(train_labels),len(val_sentences), len(val_labels),

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* `Tokenization`- direct mapping of token ( a token could be a workd or a character) to number.
* `Embedding` - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=max_length, # how long do you want your sequence to be
                                    pad_to_max_tokens=True)

In [16]:
len(train_sentences[0].split())

7

In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Setup text vactorize variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g how many words from a tweet does model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street~ "
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)

In [22]:
print(f"Orignal text: \n {random_sentence}\
      n\nVectorized Version:")
text_vectorizer([random_sentence])

Orignal text: 
 @random_tourist it rained. Some guys tree hit by lightning and some jackholes drove onto flooded streets.      n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9362,   15, 4811,   82,  576,  979,  244,   18,  300,    7,   82,
           1, 3045, 1626, 2995]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represnted by a vector 100 long
* `input_length` = length of the sequence being passed to the embedding layer

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # output shape
                             embeddings_initializer="uniform",
                            input_length=max_length # how long is each input
                            )

embedding

In [25]:
# get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Orignal text: \n {random_sentence}\
      \n\nEmbedding Version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal text: 
 In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!      

Embedding Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01533873, -0.00918882, -0.00786216, ..., -0.03983569,
         -0.02307764,  0.02726023],
        [-0.04219767,  0.04396954,  0.00230615, ..., -0.0375497 ,
         -0.0254814 , -0.00296867],
        [-0.04047302,  0.04353908, -0.02239157, ...,  0.00730013,
         -0.00166383, -0.00034189],
        ...,
        [ 0.00277431, -0.01595728, -0.02429298, ...,  0.03533996,
          0.02831711, -0.03299092],
        [-0.04219767,  0.04396954,  0.00230615, ..., -0.0375497 ,
         -0.0254814 , -0.00296867],
        [-0.01295378, -0.0230998 , -0.00807149, ...,  0.0334501 ,
          0.04490221,  0.00643456]]], dtype=float32)>

In [26]:
# Checkout a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01533873, -0.00918882, -0.00786216, -0.01781993, -0.03668386,
         0.03277623,  0.02240951, -0.03470323, -0.04869919,  0.02648507,
         0.02119419,  0.02401365,  0.00652224,  0.0249781 ,  0.04632894,
        -0.00482814, -0.00113418,  0.01254474,  0.00414731, -0.03071289,
         0.01411411,  0.03830792, -0.00982008,  0.04153406,  0.00530423,
         0.01732514, -0.04593551,  0.00535412,  0.03950068,  0.03591608,
         0.03390763,  0.03844709, -0.01270702,  0.02753779, -0.01702363,
         0.04761786,  0.0453898 ,  0.00676734, -0.00181264,  0.02572023,
        -0.01497597,  0.01007898,  0.02839608,  0.01686385, -0.01445469,
        -0.00508584, -0.01128819, -0.045731  ,  0.03520324,  0.04710246,
         0.0091259 ,  0.01851389,  0.03736389,  0.01894644,  0.03754351,
         0.04606725, -0.00200849,  0.00527537, -0.04894684, -0.02571318,
        -0.04168401, -0.04272784, -0.01707544,  0.02317408, -0.01934173,
  

## Modelling a text dataset (running series of experiments)

No we've got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We've start with a baseline and move on from there

* `Model 0`: Naive Bayes (baseline), this is from SkLearn ML Map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* `Model 1`: Feed-forward neural network (dense model)
* `Model 2`: LSTM Model (RNN)
* `Model 3`: GRU Model (RNN)
* `Model 4`: Bidirectional-LSTM Model (RNN)
* `Model 5`: 1D Convolutional Neural Network (CNN)
* `Model 6`: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* `Model 7`: Same as model 6 with 10% of training data

How are we going to approch all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model


### Model 0: Getting a baseline

as with all machine learning modelling experiments, it's important to create  a baseline model so you've got b benchmark for future eperiments to build a model

To create a baseline, we'll yse SkLearn's multinominal Naive Bayes using the TensorFlow TF-IDF formula to convert our words to numbers

> 🔑 **Note:** It's common practice to use non-DL algorithm as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [29]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's prediction with different metrics everytime, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [30]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weightted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                   "recall": model_recall,
                   "F1": model_f1 }
  return model_results

In [31]:
# Get basedline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1': 0.7862189758049549}

### Model 1: A Simple Dense Model

In [32]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [33]:
# Build model with the Funtional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # input are 1-dimentional string
x = text_vectorizer(inputs) ## turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation function

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [36]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230813-094837
Epoch 1/5
215/215 [==============================] - 21s 75ms/step - loss: 0.6107 - accuracy: 0.6910 - val_loss: 0.5383 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4422 - accuracy: 0.8178 - val_loss: 0.4703 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3474 - accuracy: 0.8616 - val_loss: 0.4615 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2852 - accuracy: 0.8886 - val_loss: 0.4719 - val_accuracy: 0.7979
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2382 - accuracy: 0.9124 - val_loss: 0.4826 - val_accuracy: 0.7822


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4826 - accuracy: 0.7822


[0.4826182425022125, 0.7821522355079651]

In [38]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
# Look at the single prediction
model_1_pred_probs[0]

array([0.3017673], dtype=float32)

In [40]:
# Look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.3017673 ],
       [0.79247874],
       [0.9979711 ],
       [0.13070434],
       [0.0984716 ],
       [0.9401178 ],
       [0.90404564],
       [0.99274784],
       [0.9618223 ],
       [0.28530324]], dtype=float32)

In [41]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
# Caculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7868451603977311,
 'recall': 0.7821522309711286,
 'F1': 0.779088324447517}

In [43]:
# Looks like our baseline model is out performing our first deep learning model
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [48]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [50]:
# Get the weeight matrix of embedding layer
# (these are the numerical represntation of each token in our training data, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim (output dim of our embedding layer)

(10000, 128)

In [51]:
embed_weights

array([[ 0.00354234, -0.00110694, -0.0320929 , ...,  0.02175982,
        -0.01791585, -0.01426832],
       [-0.04414389,  0.009406  ,  0.03783568, ..., -0.00689363,
        -0.01671204, -0.04834564],
       [-0.02842291, -0.02296582, -0.00285087, ...,  0.01976018,
         0.00983215,  0.01068427],
       ...,
       [ 0.04650222,  0.03712967,  0.03530372, ...,  0.04821125,
        -0.02289379, -0.0463891 ],
       [-0.07083952, -0.00160929, -0.00545473, ...,  0.03875023,
        -0.01337057, -0.03061764],
       [-0.04774535, -0.05113614, -0.10559413, ...,  0.1067627 ,
        -0.07219938, -0.11362486]], dtype=float32)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector : http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on workds embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [52]:
# Create embedding filrs (we got this from TensorFlow's word embeddings documentations)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [53]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long Short Term Memory (one of the most popular LSTM cells)

Our structure of RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability
```

In [54]:
# Create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [55]:
# Get a summay
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [56]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [57]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                      "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230813-095200
Epoch 1/5
215/215 [==============================] - 13s 44ms/step - loss: 0.2223 - accuracy: 0.9221 - val_loss: 0.5610 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1584 - accuracy: 0.9431 - val_loss: 0.5558 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1273 - accuracy: 0.9531 - val_loss: 0.6396 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1098 - accuracy: 0.9569 - val_loss: 0.8731 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0870 - accuracy: 0.9657 - val_loss: 0.9237 - val_accuracy: 0.7677


In [58]:
# Make prediction with LSTM model
model_2_preds_probs = model_2.predict(val_sentences)
model_2_preds_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.01352511],
       [0.6548004 ],
       [0.9998685 ],
       [0.03936565],
       [0.00146339],
       [0.99970335],
       [0.99406505],
       [0.9999069 ],
       [0.9998549 ],
       [0.688351  ]], dtype=float32)

In [59]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [60]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.77165354330708,
 'precision': 0.7681410880728078,
 'recall': 0.7677165354330708,
 'F1': 0.7662770891654436}

## Model 3: GRU

Gated Recurrent Unit

Another poweful and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less paraments

In [61]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack the recurrent layers on the top of each others, you need return_sequence=True
# print(x.shape)
# x = layers.LSTM(42, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [62]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [63]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [64]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230813-095222
Epoch 1/5
215/215 [==============================] - 12s 47ms/step - loss: 0.1551 - accuracy: 0.9419 - val_loss: 0.8810 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0856 - accuracy: 0.9683 - val_loss: 0.7699 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0717 - accuracy: 0.9711 - val_loss: 0.9112 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0672 - accuracy: 0.9712 - val_loss: 1.0410 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0540 - accuracy: 0.9769 - val_loss: 1.0681 - val_accuracy: 0.7743


In [65]:
# Make some prediction with ouyt GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.9583940e-03],
       [8.0689448e-01],
       [9.9983203e-01],
       [6.7142449e-02],
       [8.9574984e-05],
       [9.9945289e-01],
       [7.1441239e-01],
       [9.9994028e-01],
       [9.9985898e-01],
       [9.0330386e-01]], dtype=float32)

In [66]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [67]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.42782152230971,
 'precision': 0.7755035361291913,
 'recall': 0.7742782152230971,
 'F1': 0.7724787248794153}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an english sentence) however, a bidirectional RNN goes from right to left as well as left to right

In [68]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# print(x.shape)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [69]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [70]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [71]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230813-095244
Epoch 1/5
215/215 [==============================] - 15s 54ms/step - loss: 0.1091 - accuracy: 0.9673 - val_loss: 0.9243 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0508 - accuracy: 0.9771 - val_loss: 1.3842 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0457 - accuracy: 0.9803 - val_loss: 1.2666 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0404 - accuracy: 0.9810 - val_loss: 1.6423 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0381 - accuracy: 0.9826 - val_loss: 1.3464 - val_accuracy: 0.7730


In [72]:
# Make predictions with out bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[8.6069653e-05],
       [7.3208857e-01],
       [9.9998748e-01],
       [2.6866952e-01],
       [8.3758041e-06],
       [9.9927360e-01],
       [9.3963814e-01],
       [9.9999535e-01],
       [9.9998951e-01],
       [9.8504364e-01]], dtype=float32)

In [73]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [74]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.29658792650919,
 'precision': 0.7763206990944375,
 'recall': 0.7729658792650919,
 'F1': 0.7701512125975899}

## Convolutional Neral network for Text (and other types of sequences)

We've used CNNs for images are typically 2D (height x xwidth)... however, our text data is 1D

Previously we've Conv2D for our image data but now we're going to use Conv1D

The typicall structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (Typically Conv1D + pooling) -> Outputs (class probabilities)
```

In [75]:
### Model 5: Conv1D

In [76]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        strides=1, # default = "valid", the output is smaller then the input shape, "same" means output is same shape as input
                        activation="relu",
                        padding="valid") # default

conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [77]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04963643, -0.00225354, -0.10031258, ...,  0.03238753,
         -0.04237204, -0.03627233],
        [-0.01225193, -0.02526564, -0.02205112, ...,  0.04913848,
          0.01751498, -0.01357644],
        [-0.01602457, -0.04029006, -0.0125583 , ..., -0.02536049,
         -0.00594882,  0.03006981],
        ...,
        [ 0.00885139, -0.00029536, -0.02182583, ...,  0.0235039 ,
          0.00345123,  0.00101769],
        [ 0.00885139, -0.00029536, -0.02182583, ...,  0.0235039 ,
          0.00345123,  0.00101769],
        [ 0.00885139, -0.00029536, -0.02182583, ...,  0.0235039 ,
          0.00345123,  0.00101769]]], dtype=float32)>

In [78]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , 0.        , 0.00149279,
         0.        , 0.05816419, 0.        , 0.00445475, 0.        ,
         0.        , 0.        , 0.04669026, 0.04335115, 0.00587709,
         0.02850443, 0.02420432, 0.        , 0.        , 0.02099296,
         0.01494234, 0.        , 0.        , 0.06725155, 0.        ,
         0.0170718 , 0.00709304, 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.01819921, 0.        , 0.00617332, 0.        , 0.03190684,
         0.        , 0.01510456, 0.        , 0.0769642 , 0.        ,
         0.        , 0.        , 0.07687177, 0.04685963, 0.        ,
         0.00313572, 0.        , 0.03148254, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.10990226, 0.03577044,
         0.        , 0.        , 0.03390248, 0.        , 0.04276898,
         0.        , 0.        ],
        [0.        , 0.        , 0.02807684, 0.0017

In [79]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.01819921, 0.        , 0.02807684, 0.00175239, 0.03190684,
        0.02710495, 0.07306882, 0.05293982, 0.0769642 , 0.06473447,
        0.04052504, 0.00408508, 0.07687177, 0.04685963, 0.06169479,
        0.03361366, 0.02420432, 0.03148254, 0.        , 0.02099296,
        0.01494234, 0.02321136, 0.05328   , 0.10990226, 0.03577044,
        0.03201013, 0.00709304, 0.03390248, 0.06063167, 0.08305568,
        0.01163786, 0.01109536]], dtype=float32)>

In [80]:
# Create 1-dimesional convolutional layer to model sequences
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [81]:
# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [82]:
# Get a summary of our Conv1d model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [83]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230813-095313
Epoch 1/5
215/215 [==============================] - 12s 45ms/step - loss: 0.1200 - accuracy: 0.9596 - val_loss: 0.9129 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0738 - accuracy: 0.9734 - val_loss: 1.1160 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0620 - accuracy: 0.9745 - val_loss: 1.1470 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0541 - accuracy: 0.9781 - val_loss: 1.1641 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0513 - accuracy: 0.9788 - val_loss: 1.2018 - val_accuracy: 0.7664


In [84]:
# Make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.4404374e-01],
       [7.3188436e-01],
       [9.9994946e-01],
       [6.5038994e-02],
       [3.0441629e-07],
       [9.9892223e-01],
       [9.9594241e-01],
       [9.9998891e-01],
       [9.9999976e-01],
       [9.6165091e-01]], dtype=float32)

In [85]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [86]:
# Evaluate model 5 predictions
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)

model_5_results

{'accuracy': 76.64041994750657,
 'precision': 0.7664041994750657,
 'recall': 0.7664041994750657,
 'F1': 0.7664041994750657}

## Model 6: TensorFlowHub PreTrained Feature Extractor

Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder : https://tfhub.dev/google/universal-sentence-encoder/4

In [87]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turnes into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.00477086  0.0293698   0.06304868 -0.00976107  0.03536275  0.05988206
  0.04076201  0.05354147 -0.00170396  0.01987924  0.01672826  0.00499968
  0.02665901  0.08346527  0.07317044 -0.05286197  0.03112919 -0.055748
  0.03831363 -0.0400802  -0.03261067  0.04829443  0.04296905  0.02417224
 -0.01937103 -0.01896992 -0.01253994  0.02908366 -0.04963567 -0.02397985
 -0.05901828  0.01279296 -0.02139725 -0.03211551 -0.00769663 -0.06545766
  0.00214907  0.04122009 -0.01750335 -0.07401229 -0.00822927 -0.0333032
 -0.0265524   0.05162273 -0.11012802 -0.01490786 -0.01965857  0.00943312
 -0.02796747  0.03919587], shape=(50,), dtype=float32)


In [88]:
embed_samples[0].shape

TensorShape([512])

In [89]:
sample_sentence

"There's a flood in my street~ "

In [90]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [107]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [108]:
# Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [109]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [110]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=6,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentences_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentences_encoder/20230813-105743
Epoch 1/6
215/215 [==============================] - 4s 13ms/step - loss: 0.5045 - accuracy: 0.7860 - val_loss: 0.4509 - val_accuracy: 0.8005
Epoch 2/6
215/215 [==============================] - 2s 10ms/step - loss: 0.4149 - accuracy: 0.8161 - val_loss: 0.4378 - val_accuracy: 0.8123
Epoch 3/6
215/215 [==============================] - 2s 11ms/step - loss: 0.4017 - accuracy: 0.8224 - val_loss: 0.4319 - val_accuracy: 0.8136
Epoch 4/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3928 - accuracy: 0.8269 - val_loss: 0.4326 - val_accuracy: 0.8150
Epoch 5/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3864 - accuracy: 0.8305 - val_loss: 0.4265 - val_accuracy: 0.8176
Epoch 6/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3810 - accuracy: 0.8278 - val_loss: 0.4287 - val_accuracy: 0.8163


In [111]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 7ms/step


array([[0.14421551],
       [0.77255744],
       [0.9918612 ],
       [0.1929218 ],
       [0.7244591 ],
       [0.73311067],
       [0.9819733 ],
       [0.98357844],
       [0.94529134],
       [0.0846777 ]], dtype=float32)

In [112]:
# Convert prediction probabilites to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [113]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.62729658792651,
 'precision': 0.8187546712946793,
 'recall': 0.8162729658792651,
 'F1': 0.8147089025083661}

In [114]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1': 0.7862189758049549}

## Model 7: TF Hub Pretrained USE but with 10% of trainin data

Transfer Learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data

In [117]:
# Create subsets of 10% of the training data
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)

train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()

len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [118]:
# Check the number of target in our subset of data
train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

In [119]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [121]:
# Create model using the Sequential API
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [122]:
# Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [123]:
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [125]:
# Train a classifier on top of USE pretrained embeddings
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_senteces_encoder_10_percent")])

Saving TensorBoard log files to: model_logs/tf_hub_senteces_encoder_10_percent/20230813-112559
Epoch 1/5
24/24 [==============================] - 2s 36ms/step - loss: 0.6624 - accuracy: 0.7135 - val_loss: 0.6225 - val_accuracy: 0.7913
Epoch 2/5
24/24 [==============================] - 1s 23ms/step - loss: 0.5886 - accuracy: 0.7898 - val_loss: 0.5422 - val_accuracy: 0.8005
Epoch 3/5
24/24 [==============================] - 1s 22ms/step - loss: 0.5147 - accuracy: 0.7976 - val_loss: 0.4778 - val_accuracy: 0.8031
Epoch 4/5
24/24 [==============================] - 1s 24ms/step - loss: 0.4608 - accuracy: 0.8068 - val_loss: 0.4358 - val_accuracy: 0.8255
Epoch 5/5
24/24 [==============================] - 1s 24ms/step - loss: 0.4271 - accuracy: 0.8239 - val_loss: 0.4065 - val_accuracy: 0.8320


In [128]:
# Make predictions with USE TF Hub model 10 percent
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 7ms/step


array([[0.2591904 ],
       [0.8138575 ],
       [0.9101021 ],
       [0.29470918],
       [0.80551845],
       [0.8336647 ],
       [0.9108048 ],
       [0.9361573 ],
       [0.8545173 ],
       [0.06909795]], dtype=float32)

In [129]:
# Convert prediction probabilities to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [133]:
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)

model_7_results

{'accuracy': 83.2020997375328,
 'precision': 0.831933344290887,
 'recall': 0.8320209973753281,
 'F1': 0.831693372736778}